In [1]:
import random
import pandas as pd
random.seed(1018)

# Define commodity types
commodity_types = ['Electronic', 'Mechanical', 'Chemical']

# Define criticality levels with weights
criticality_levels = ['High', 'Medium', 'Low']
criticality_weights = [2, 3, 4]  # Weights ensure fewer high criticality parts and more low criticality parts

parts = [
    {"product_number": 1, "product_description": "Iron Man Suit", "commodity_type": "Mechanical", "criticality": "High"},
    {"product_number": 7, "product_description": "Flux Capacitor", "commodity_type": "Electronic", "criticality": "High"},
    {"product_number": 2, "product_description": "Antikythera Mechanism", "commodity_type": "Mechanical", "criticality": "Medium"},
    {"product_number": 4, "product_description": "Darksaber", "commodity_type": "Electronic", "criticality": "Medium"},
    {"product_number": 5, "product_description": "Omni Tool", "commodity_type": "Electronic", "criticality": "Low"},
    {"product_number": 6, "product_description": "Batarang", "commodity_type": "Mechanical", "criticality": "Low"},
    {"product_number": 8, "product_description": "Silmaril", "commodity_type": "Mechanical", "criticality": "Low"},
    {"product_number": 9, "product_description": "Tricorder", "commodity_type": "Electrical", "criticality": "Low"},
    {"product_number": 3, "product_description": "Pip-Boy", "commodity_type": "Electrical", "criticality": "Low"},
    {"product_number": 10, "product_description": "Super Soldier Serum", "commodity_type": "Chemical", "criticality": "Medium"}
]


# Convert to DataFrame for display
parts_df = pd.DataFrame(parts)

parts_df


,product_number,product_description,commodity_type,criticality
0,1,Iron Man Suit,Mechanical,High
1,7,Flux Capacitor,Electronic,High
2,2,Antikythera Mechanism,Mechanical,Medium
3,4,Darksaber,Electronic,Medium
4,5,Omni Tool,Electronic,Low
5,6,Batarang,Mechanical,Low
6,8,Silmaril,Mechanical,Low
7,9,Tricorder,Electrical,Low
8,3,Pip-Boy,Electrical,Low
9,10,Super Soldier Serum,Chemical,Medium


In [2]:
from datetime import timedelta, datetime

# Set random seed for reproducibility
random.seed(42)

# Vendors and their respective criticality
vendors = {
    "Acme Corporation": "High",
    "Erebor Heavy Industries": "Medium",
    "Wayne Enterprises": "Low",
    "Vault-Tec LLT": "Low",
    "Weyland-Yutani Corp": "Low"
}

criticality_levels = ['High', 'Medium', 'Low']

# Filter parts by criticality
def filter_parts_by_criticality(criticality):
    return [part for part in parts if part['Criticality'] == criticality]

# Function to generate random dates for orders
def random_date(start, end):
    return start + timedelta(days=random.randint(0, (end - start).days))

# Generate orders
num_orders = 1000
orders = []

for i in range(num_orders):
    
    part = random.choices(parts, weights=[10,10,20,20,30,30,30,30,30,30])[0]
    criticality = part['criticality']
    #vendor = random.choices(list(vendors.keys()), weights=[20,20,25,30,30], k=1)[0]
    # criticality = vendors[vendor]  # Vendor-specific criticality
    if criticality == "High":
        vendor = random.choices(list(vendors.keys()), weights=[4,1,0,0,0 ], k=1)[0]
    elif criticality == 'Medium':
        vendor = random.choices(list(vendors.keys()), weights=[2,6,6,1,1 ], k=1)[0]
    else:
        vendor = random.choices(list(vendors.keys()), weights=[1,1,5,10,10 ], k=1)[0]

    order_number = f"PO-{i+1:03}"
    quantity = random.randint(10, 100)
    total_order_due_date = random_date(datetime(2024, 1, 1), datetime(2024, 11, 30))

    order_cost = round(quantity * random.uniform(5, 50), 2)
    
    orders.append({
        "order_number": order_number,
        "vendor": vendor,
        "product_number": part["product_number"],
        #"Product Name": part["Product Name"],
        #"Commodity Type": part["Commodity Type"],
        #"Criticality": criticality,
        "quantity": quantity,
        "due_date": total_order_due_date,
        #"Line Item Due Date": line_item_due_date,
        #"Order Cost": order_cost
    })

# Convert orders to DataFrame
orders_df = pd.DataFrame(orders)
orders_df.describe()

,product_number,quantity,due_date
count,1000.000000,1000.000000,1000
mean,5.922000,55.006000,2024-06-20 06:50:23.999999744
min,1.000000,10.000000,2024-01-01 00:00:00
25%,3.000000,32.000000,2024-03-31 18:00:00
50%,6.000000,54.000000,2024-06-24 00:00:00
75%,8.000000,79.000000,2024-09-09 00:00:00
max,10.000000,100.000000,2024-11-30 00:00:00
std,2.756722,26.319587,NaN


In [3]:
import pandas as pd
import random
from datetime import timedelta, datetime

# Set random seed for reproducibility

# Function to generate random dates for receipts
def random_date(order_date, low_delta, high_delta, late_weights=None):

    late_flag = random.choices([True, False], weights=late_weights)[0]
    
    if late_flag:
        delta = random.randint(1, high_delta+1)
        return order_date + timedelta(days=delta)
    else:
        delta = random.randint(0, low_delta+1)
        return order_date - timedelta(days=delta)

# Generate Order Receipts
order_receipts = []

for index, order in orders_df.iterrows():
    order_quantity = order['quantity']
    total_received = 0

    # Generate between 1 and 4 shipments per order
    num_shipments = random.randint(1, 8)

    for i in range(num_shipments):
        # Ensure the total quantity received doesn't exceed the ordered quantity
        if total_received >= order_quantity:
            break

        # Randomly determine the quantity received for this shipment
        max_receivable_quantity = order_quantity - total_received
        quantity_received = random.randint(1, max_receivable_quantity)

        # Generate a random received date after the order was placed
        if order['vendor'] == 'Weyland-Yutani Corp': # 10% Late but by a lot
            received_date = random_date(order['due_date'],low_delta=5,high_delta=20,late_weights=[1,9])
        elif order['vendor'] == 'Acme Corporation': # rarely late,
            received_date = random_date(order['due_date'],low_delta=10,high_delta=5,late_weights=[1,99])
        elif order['vendor'] == 'Vault-Tec LLT': # 15% often late, never early
            received_date = random_date(order['due_date'],low_delta=0,high_delta=5,late_weights=[15,85])
        elif order['vendor'] == 'Wayne Enterprises': # Often early, rarely late
            received_date = random_date(order['due_date'],low_delta=20,high_delta=1,late_weights=[1,49])
        elif (order['vendor'] == 'Erebor Heavy Industries') & (order['due_date'] < datetime(2024, 10, 1)):
            received_date = random_date(order['due_date'],low_delta=10,high_delta=10,late_weights=[20,80])
        else:
            received_date = random_date(order['due_date'],low_delta=1,high_delta=10,late_weights=[95,5])
            

#        received_date = random_date(order['due_date'], datetime(2024, 12, 31))

        # Append receipt details
        order_receipts.append({
            "order_number": order['order_number'],
            "vendor" : order['vendor'],
            #"Product Number": order['Product Number'],
            #"Quantity Ordered": order_quantity,
            "quantity_received": quantity_received,
            "received_date": received_date
        })

        # Update total received quantity
        total_received += quantity_received

# Convert order receipts to DataFrame
order_receipts_df = pd.DataFrame(order_receipts)
order_receipts_df = order_receipts_df.sort_values('received_date').reset_index(drop=True)
order_receipts_df

,order_number,vendor,quantity_received,received_date
0,PO-909,Wayne Enterprises,2,2023-12-16
1,PO-939,Wayne Enterprises,4,2023-12-16
2,PO-909,Wayne Enterprises,30,2023-12-17
3,PO-755,Wayne Enterprises,85,2023-12-17
4,PO-023,Wayne Enterprises,42,2023-12-19
...,...,...,...,...
3390,PO-478,Vault-Tec LLT,1,2024-12-03
3391,PO-833,Vault-Tec LLT,5,2024-12-03
3392,PO-144,Weyland-Yutani Corp,1,2024-12-04
3393,PO-393,Weyland-Yutani Corp,2,2024-12-10


In [4]:

# Function to generate a random inspection date within 14 days of the received date
def random_inspection_date(received_date):
    return received_date + timedelta(days=random.randint(0, 14))

# Define possible defect codes
defect_codes = ['Undersized / Underweight', 'Cosmetic Damage', 'Labeled Incorrectly', 'Non-Functioning', 'Physical Damage']

# Generate Receiving Inspection Data
inspection_data = []

for index, receipt in order_receipts_df.iterrows():
    received_quantity = receipt["quantity_received"]
    received_date = receipt["received_date"]
    
    # Generate an inspection date
    inspection_date = random_inspection_date(received_date)
    
    # Randomly decide if there's a defect with a 1 in 10 chance
    if receipt['vendor'] == 'Weyland-Yutani Corp':
        defect = random.choices([True, False], weights=[2, 8], k=1)[0]
    elif receipt['vendor'] == 'Acme Corporation':
        defect = random.choices([True, False], weights=[1, 19], k=1)[0]
    elif receipt['vendor'] == 'Vault-Tec LLT':
        defect = random.choices([True, False], weights=[10, 10], k=1)[0]
    else:
        defect = random.choices([True, False], weights=[1, 9], k=1)[0]
    
    if defect:
        # If there is a defect, the inspected quantity equals the received quantity
        inspected_quantity = received_quantity
        # Randomly reject between 1 to the total received quantity
        if receipt['vendor'] == 'Weyland-Yutani Corp':
            rejected_quantity = random.randint(received_quantity//2+1, received_quantity)
        elif receipt['vendor'] == 'Acme Corporation':
            rejected_quantity = random.randint(1, received_quantity//3+1)
        elif receipt['vendor'] == 'Vault-Tec LLT':
            rejected_quantity = random.randint(received_quantity//3+1, received_quantity)
        else:
            rejected_quantity = random.randint(1, received_quantity)

        # rejected_quantity = random.randint(1, received_quantity)
        # Assign a random defect code from the list
        defect_code = random.choice(defect_codes)
    else:
        # If no defect, inspect a random portion or all the received quantity
        inspected_quantity = (received_quantity // 10 ) if received_quantity > 10 else received_quantity
        rejected_quantity = 0
        # No defect, so defect code is None
        defect_code = None

    accepted_quantity = received_quantity - rejected_quantity
    
    # Append inspection details
    inspection_data.append({
        "receipt_number": index +1,
        "inspection_date": inspection_date,
        "quantity_inspected": inspected_quantity,
        "quantity_accepted": accepted_quantity,
        "quantity_rejected": rejected_quantity,
        "rejected": defect,
        "defect_code": defect_code,
        #"Vendor": receipt['Vendor']
    })


# Convert to DataFrame
inspection_df = pd.DataFrame(inspection_data)
inspection_df #= inspection_df.sort_values('Inspection Date').reset_index(drop=True)

,receipt_number,inspection_date,quantity_inspected,quantity_accepted,quantity_rejected,rejected,defect_code
0,1,2023-12-26,2,2,0,False,None
1,2,2023-12-21,4,4,0,False,None
2,3,2023-12-22,3,30,0,False,None
3,4,2023-12-18,8,85,0,False,None
4,5,2024-01-01,4,42,0,False,None
...,...,...,...,...,...,...,...
3390,3391,2024-12-09,1,0,1,True,Labeled Incorrectly
3391,3392,2024-12-09,5,5,0,False,None
3392,3393,2024-12-10,1,0,1,True,Cosmetic Damage
3393,3394,2024-12-21,2,2,0,False,None


In [5]:
# Insert DataFrame data into the SQLite table
order_receipts_df = order_receipts_df.drop(columns='vendor')
df_list = [parts_df, orders_df, order_receipts_df, inspection_df]
table_names = ['items', 'orders', 'receipts', 'inspections']


# Insert DataFrame data into the SQLite table

import sqlite3

# Create a connection to the SQLite database (it will create a new one if not exists)
conn = sqlite3.connect('synthetic_erp.db')

# Create a cursor object
cursor = conn.cursor()

try:
    for df, table in zip(df_list,table_names):

        df.index = df.index + 1
        df = df.reset_index().rename(columns={'index': 'id'})
        df.to_sql(table, conn, if_exists='replace', index=False )
        conn.commit()
        # df.to_sql('users', conn, if_exists='replace', index=False)

        # # Commit and close connection
        # conn.commit()
        # conn.close()

        # print("Data inserted successfully into SQLite database.")
finally:
    cursor.close() 
    conn.close()

In [6]:
conn.close()
